In [1]:
import pandas as pd
import requests
import json
from time import sleep
from functools import reduce

In [2]:
ids = pd.read_csv('../../../archive/high_diamond_ranked_10min_no_identical.csv')['gameId'][:100]

In [3]:
ids

0     4519157822
1     4523371949
2     4521474530
3     4524384067
4     4436033771
         ...    
95    4524217548
96    4499622849
97    4499844894
98    4460534353
99    4495269641
Name: gameId, Length: 100, dtype: int64

In [4]:
champions_list = []
champ_item_list = []
champ_levels_list = []
for id_ in ids:
    url = f'https://euw1.api.riotgames.com/lol/match/v4/timelines/by-match/{id_}?api_key=RGAPI-9416670c-0f0d-4381-b0a9-c9e723634458'
    got = requests.get(url)
    timeline = json.loads(got.text)
    sleep(1.2)
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matches/{id_}?api_key=RGAPI-9416670c-0f0d-4381-b0a9-c9e723634458'
    got = requests.get(url)
    match = json.loads(got.text)
    sleep(1.2)
    
    participants = match['participants']
    
    champs = [(p['participantId'], p['teamId'], p['championId'],
               p['timeline']['role'], p['timeline']['lane']) for p in participants]
    champions_list.append(champs)
    
    frames = timeline['frames'][:11]
    events = [frame['events'] for frame in frames]
    events = reduce(lambda a, b: a+b, events)
    item_events = [event for event in events if event['type'].startswith('ITEM')]
    
    champ_items = {i:[] for i in range(1, 11)}
    for event in item_events:
        participant = event['participantId']
        type_ = event['type']
        if type_ == 'ITEM_UNDO':
            if event['beforeId'] != 0:
                champ_items[participant].remove(event['beforeId'])
            else:
                champ_items[participant].append(event['afterId'])
        elif type_ == 'ITEM_PURCHASED':
            item = event['itemId']
            champ_items[participant].append(item)
    
    champ_item_list.append(champ_items)
    
    final = frames[-1]['participantFrames']
    levels = champ_items = {i:None for i in range(1, 11)}
    
    for participant in final.values():
        levels[participant['participantId']] = participant['level']
    
    champ_levels_list.append(levels)

In [5]:
df = pd.DataFrame({'champions':champions_list, 'items':champ_item_list, 'levels': champ_levels_list})
df.head()

,champions,items,levels
0,"[(1, 100, 111, DUO_SUPPORT, BOTTOM), (2, 100, ...","{1: [3858, 2003, 3340, 1001, 2003, 1029, 2003,...","{1: 5, 2: 7, 3: 7, 4: 7, 5: 7, 6: 6, 7: 6, 8: ..."
1,"[(1, 100, 4, DUO_SUPPORT, NONE), (2, 100, 350,...","{1: [3340, 2003, 1056, 2003, 1052, 1001, 2003,...","{1: 7, 2: 5, 3: 8, 4: 6, 5: 7, 6: 6, 7: 7, 8: ..."
2,"[(1, 100, 421, NONE, JUNGLE), (2, 100, 84, SOL...","{1: [1041, 2031, 3340, 3364, 1036, 3706, 1036,...","{1: 6, 2: 8, 3: 5, 4: 6, 5: 7, 6: 6, 7: 6, 8: ..."
3,"[(1, 100, 64, NONE, JUNGLE), (2, 100, 134, SOL...","{1: [1041, 2031, 3340, 3715, 2055], 2: [1056, ...","{1: 7, 2: 8, 3: 7, 4: 7, 5: 6, 6: 6, 7: 6, 8: ..."
4,"[(1, 100, 101, DUO, TOP), (2, 100, 266, DUO, T...","{1: [1056, 2003, 2003, 3340, 1082, 1001, 2055,...","{1: 8, 2: 8, 3: 7, 4: 5, 5: 7, 6: 6, 7: 8, 8: ..."
